<a href="https://colab.research.google.com/github/iamrishav111/Spell-Checker-Model/blob/master/SpellCheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
import numpy as np
import itertools


right = []
wrong = []
word2dict = {}


with open('spell-errors.txt') as f:
  for line in f:
    right_words = line.split(':')
    right.append(right_words[0])
    wrong_words = right_words[1].split(',')
   
    wrong.append(wrong_words)
    word2dict[right_words[0]]  = wrong_words

alphabet = 'abcdefghijklmnopqrstuvwxyz -'

def split(word): 
    return [char for char in word] 

In [ ]:
def count(w1):
  counter = 0
  length = len(w1)
  for word in (right+wrong):
    for i in range(len(word)-length):
      if word[i:i+length] == w1:
        counter += 1
  return counter

In [ ]:
word_dict = {}
for i,char in enumerate(alphabet):
  word_dict[char] = i
  
  

In [ ]:
def edittype(candidate,word):
  edit = [False]*4
  correct = ''
  error = ''
  x = ''
  y = ''
  for i in range(min([len(word),len(candidate)])-1):
    if candidate[0:i+1] != word[0:i+1]:
      if candidate[i:] == word[i-1:]:
        edit[1] = True
        x = candidate[i-2]
        w = candidate[i-2]+candidate[i-1]
        break
      elif candidate[i:] == word[i+1:]:
        if i == 0:
          
          w = '_'
          x = '_'+word[i]
        else:
          
          w = word[i-1]
          x = word[i-1]+word[i]
        edit[0] = True
        break
      if candidate[i+1:] == word[i+1:]:
        
        edit[2] = True
        x = word[i]
        w = candidate[i]
        break
      if candidate[i] == word[i+1] and candidate[i+2:] == word[i+2:]:
        
        edit[3] = True
        w = candidate[i]+candidate[i+1]
        x = word[i]+word[i+1]
        break   
  if word == candidate:
    return "not required",'',''
  if edit[1]:
    return "Deletion",x,w
  elif edit[0]:
    return "Insertion",x,w
  elif edit[2]:
    return "Substitution",x,w
  elif edit[3]:
    return "Reversal",x,w
  else :
    return "not required",'',''   


In [ ]:
sub_matrix = np.zeros((28,28))
add_matrix = np.zeros((28,28))
del_matrix = np.zeros((28,28))
trans_matrix = np.zeros((28,28))

def subconfusion_matrix(sub_matrix,x,y):
  if x in split(alphabet) and y in split(alphabet):
    
    sub_matrix[word_dict[x]][word_dict[y]] += 1
  
  
def addconfusion_matrix(add_matrix,x,y):
  if x in split(alphabet) and y in split(alphabet):
    
    add_matrix[word_dict[x]][word_dict[y]] += 1
  
  
def delconfusion_matrix(del_matrix,x,y):
  
  if x in split(alphabet) and y in split(alphabet):
    
    del_matrix[word_dict[x]][word_dict[y]] += 1
  
  
  
def transconfusion_matrix(trans_matrix,x,y):
  if x in split(alphabet) and y in split(alphabet):
    
    trans_matrix[word_dict[x]][word_dict[y]] += 1

In [ ]:
def update(type,x,w):
  
  if type == 'Deletion':
    delconfusion_matrix(del_matrix,w[0],w[1])
  elif type == 'Insertion':
    addconfusion_matrix(add_matrix,w,x[1])    
  elif type == 'Substitution':
    subconfusion_matrix(sub_matrix,x,w)    
  elif type == 'Reversal':
    transconfusion_matrix(trans_matrix,w[0],w[1])  
  

In [ ]:

for right_word in right:
  for wrong_word in word2dict[right_word]:
    new_wrong_word = wrong_word.lstrip()
    wrong_strip = new_wrong_word.strip('\n')
    right_strip = right_word.strip('\n')
    type,x1,w1 = edittype(right_strip,wrong_strip)
    update(type,x1,w1)    

In [ ]:
print(np.sum(del_matrix),np.sum(sub_matrix),np.sum(add_matrix),np.sum(trans_matrix))

In [ ]:
edittype('poetry','peotry')

In [ ]:
edittype('something','somthing')

In [ ]:
def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)
    

In [ ]:
edits1('rishav')

In [ ]:
def known_edits1(word):
  return set(e1 for e1 in edits1(word) if e1 in right)

In [ ]:
def prob_estimate(candidate_words,error_word):
  try:
    prob_word = {}
    for word in candidate_words:     
      type,x,w = edittype(error_word,word)
 
      if type == 'Deletion':
        if w[0] in split(alphabet) and w[1] in split(alphabet):
          count_num = del_matrix[word_dict[w[0]]][word_dict[w[1]]]
          total_count = count(w)
        
      elif type == 'Insertion':
        if x[1] in split(alphabet) and w in split(alphabet):
          count_num = add_matrix[word_dict[w]][word_dict[x[1]]]
          total_count = count(w)
                               
      elif type == 'Substitution':
        if x in split(alphabet) and w in split(alphabet):
          count_num = sub_matrix[word_dict[x]][word_dict[w]] 
          total_count = count(w)
                             
      elif type == 'Reversal':
        if w[0] in split(alphabet) and w[1] in split(alphabet):
          count_num = trans_matrix[word_dict[w[0]]][word_dict[w[1]]]
          total_count = count(w)
                                 
      prob_word[word] = count_num/total_count
    return prob_word
  except:
    print('error occured')
  

In [ ]:
known_edits1('sommthing')

{'something'}

In [ ]:
prob_estimate(known_edits1('peotry'),'peotry')

{'poetry': 0.04878048780487805}